In [1]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip

--2022-04-26 00:45:24--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.9.2 [following]
--2022-04-26 00:45:24--  https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4.17M  14.4MB/s    in 0.3s    

2022-04-26 00:45:25 (14.4 MB/s) - ‘v0.9.2.zip’ saved [4369852]



In [2]:
! unzip v0.9.2.zip

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
   creating: fastText-0.9.2/
   creating: fastText-0.9.2/.circleci/
  inflating: fastText-0.9.2/.circleci/cmake_test.sh  
  inflating: fastText-0.9.2/.circleci/config.yml  
  inflating: fastText-0.9.2/.circleci/gcc_test.sh  
  inflating: fastText-0.9.2/.circleci/pip_test.sh  
  inflating: fastText-0.9.2/.circleci/pull_data.sh  
  inflating: fastText-0.9.2/.circleci/python_test.sh  
  inflating: fastText-0.9.2/.circleci/run_locally.sh  
  inflating: fastText-0.9.2/.circleci/setup_circleimg.sh  
  inflating: fastText-0.9.2/.circleci/setup_debian.sh  
  inflating: fastText-0.9.2/.gitignore  
  inflating: fastText-0.9.2/CMakeLists.txt  
  inflating: fastText-0.9.2/CODE_OF_CONDUCT.md  
  inflating: fastText-0.9.2/CONTRIBUTING.md  
  inflating: fastText-0.9.2/LICENSE  
  inflating: fastText-0.9.2/MANIFEST.in  
  inflating: fastText-0.9.2/Makefile  
  inflating: fastText-0.9.2/README.md  
   creating: fastText-0.9.2/alignment/
  in

In [3]:
! cd fastText-0.9.2
# for command line tool :
! make
# for python bindings :
! pip install fasttext

make: *** No targets specified and no makefile found.  Stop.
     |████████████████████████████████| 68 kB 3.1 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3141830 sha256=4c357a4a19f06bc8e799a4266d1592ff21c1322d3980ec79b83d15d43be66d9a
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


This notebook showcases the performance of fastText on the dataset D_Unbalanced. The model and its output can be found at the end of the notebook. 

In [4]:
#Generic libraries
import pandas as pd
import numpy as np

#Library for preprocessing
from gensim.utils import simple_preprocess

#Library for fastText
import fasttext

#Library for csv file manipulation
import csv

#sklearn modules 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [5]:
df = pd.read_csv("data_preprocessed_for_fasttext.csv", lineterminator='\n')

In [6]:
df.head()

,issue,label,text,cleaned_text
0,__label__bug `parse_tax_data` should be able t...,0,`parse_tax_data` should be able to generate de...,parse tax data should be able to generate defa...
1,__label__bug Problems with creating new custom...,0,Problems with creating new custom recipes for ...,Problems with creating new custom recipes for ...
2,__label__bug GCP Client does not put snapshot ...,0,GCP Client does not put snapshot id in output ...,GCP Client does not put snapshot id in output ...
3,__label__bug Fix Vue default object property I...,0,Fix Vue default object property IntelliSense C...,Fix Vue default object property IntelliSense C...
4,__label__bug Error message for duplicate user ...,0,Error message for duplicate user names Right ...,Error message for duplicate user names Right n...


The data we're working with here has already been cleaned and preprocessed. Along with the original data, we have the cleaned data which is what we'll be working with. 

In [7]:
df.describe()

,label
count,13000.000000
mean,0.846154
std,0.769260
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,2.000000


In [8]:
X = df['cleaned_text']
X

0        parse tax data should be able to generate defa...
1        Problems with creating new custom recipes for ...
2        GCP Client does not put snapshot id in output ...
3        Fix Vue default object property IntelliSense C...
4        Error message for duplicate user names Right n...
                               ...                        
12995    Why is SA2 by DJZ 2011 re reading? In 2481 an ...
12996    make errors last K release make clean make dis...
12997    Is it possible to have different buying strate...
12998    Should the log writer always handle it's own e...
12999    Question Usage of adUnits.sizes will eventuall...
Name: cleaned_text, Length: 13000, dtype: object

In [9]:
y = df['label']
y

0        0
1        0
2        0
3        0
4        0
        ..
12995    2
12996    2
12997    2
12998    2
12999    2
Name: label, Length: 13000, dtype: int64

In [10]:
train, test = train_test_split(df, test_size=0.2, random_state=16022022)

In [11]:
# This step is skipped as the data is already preprocessed

# train.iloc[:, 1] = train.iloc[:, 1].apply(lambda x: ' '.join(simple_preprocess(x)))
# test.iloc[:, 1] = test.iloc[:, 1].apply(lambda x: ' '.join(simple_preprocess(x)))

In [12]:
cols = ['cleaned_text', 'label']
train = train[cols]
test = test[cols]

In [13]:
print(train.columns)

Index(['cleaned_text', 'label'], dtype='object')


In [14]:
train['label']

454      0
4741     0
12934    2
828      0
6096     1
        ..
10321    2
6813     1
11670    2
3340     0
6223     1
Name: label, Length: 10400, dtype: int64

In [15]:
y_true_numbers = test['label']
y_true_numbers

5232     1
1572     0
10652    2
1386     0
2119     0
        ..
11444    2
11674    2
5549     1
4173     0
3448     0
Name: label, Length: 2600, dtype: int64

We need the ground truth values as numbers before it is converted into categorical data for McNemar's test

In [16]:
# We need the labels to be in the format "__label__xyz" for fastText's input 

mapping = {0:'__label__bug', 1:'__label__enhancement', 2:'__label__question'}
train['label'] = train['label'].apply(lambda x : mapping[x])

In [17]:
train['label']

454              __label__bug
4741             __label__bug
12934       __label__question
828              __label__bug
6096     __label__enhancement
                 ...         
10321       __label__question
6813     __label__enhancement
11670       __label__question
3340             __label__bug
6223     __label__enhancement
Name: label, Length: 10400, dtype: object

In [18]:
test['label'] = test['label'].apply(lambda x : mapping[x])

In [19]:
train.to_csv('train_fasttext.txt', index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")
test.to_csv('test_fasttext.txt', index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

In [20]:
model = fasttext.train_supervised('train_fasttext.txt', wordNgrams = 2)

model.test('test_fasttext.txt')

(2600, 0.6984615384615385, 0.6984615384615385)

In [21]:
for i in range(0,5):
  print(model.predict(test.iloc[i, 0]))

(('__label__enhancement',), array([0.59900939]))
(('__label__bug',), array([0.49014324]))
(('__label__enhancement',), array([0.67276031]))
(('__label__enhancement',), array([0.37828413]))
(('__label__bug',), array([0.50663692]))


In [22]:
preds = []
for data in test['cleaned_text']:
  preds.append(model.predict(data))

In [23]:
preds[0]

(('__label__enhancement',), array([0.59900939]))

In [24]:
testtuple = preds

print(testtuple[0][0][0].split("'),")[0])

__label__enhancement


In [25]:
preds_clean=[]
for data in preds:
  preds_clean.append(data[0][0].split("'),")[0])

In [26]:
preds_number = []

for x in preds_clean:
  if x == '__label__bug':
    preds_number.append(0)
  if x == '__label__enhancement':
    preds_number.append(1)
  if x == '__label__question':
    preds_number.append(2)

We need the predicted values as numbers for McNemar's test

In [27]:
confusion_matrix(test['label'], preds_clean)

array([[731, 188,  78],
       [179, 739,  63],
       [162, 114, 346]])

In [28]:
target_names = ['bug', 'enhancements', 'question']
print(classification_report(test['label'], preds_clean, target_names=target_names))

              precision    recall  f1-score   support

         bug       0.68      0.73      0.71       997
enhancements       0.71      0.75      0.73       981
    question       0.71      0.56      0.62       622

    accuracy                           0.70      2600
   macro avg       0.70      0.68      0.69      2600
weighted avg       0.70      0.70      0.70      2600



In [29]:
model.save_model("fasttext_model.bin")